Final queries for visualizations

In [1]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json
from calendar import monthrange
from datetime import datetime

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirm list of databases
print(mongo.list_database_names())

['admin', 'autosaurus', 'chicago_bikes', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'mongodbVSCodePlaygroundDB', 'petsitly_marketing', 'travel_db', 'uk_food']


In [4]:
# Assign the database to a variable name
db = mongo.chicago_bikes

In [5]:
# Review collections in database
print(db.list_collection_names())

['divvy_ridedata', 'Top10Routes', 'withoutStationName', 'Top10EndStations', 'withStationName', 'Top10StartStations', 'divvy_ridedata_merged', 'weather_daily']


In [6]:
# Assign collections to variables 
divvy_rides = db['divvy_ridedata']
weather_daily = db['weather_daily']
divvy_ridedata_merged = db['divvy_ridedata_merged']
withoutStation = db['withoutStationName']
withStation = db['withStationName']
withLatLong = db['withLatLong']

In [7]:
# Count the number of documents in the collections 
print(divvy_rides.count_documents({}))
print(weather_daily.count_documents({}))
print(divvy_ridedata_merged.count_documents({}))
print(withoutStation.count_documents({}))
print(withStation.count_documents({}))
print(withLatLong.count_documents({}))

5435346
365
5435346
397105
4204965
0


In [8]:
# Review a document in each collection 
print(db.divvy_ridedata.find_one())
print(db.weather_daily.find_one())
print(db.divvy_ridedata_merged.find_one())
print(db.withoutStationName.find_one())
print(db.withStationName.find_one())
print(db.withLatLong.find_one())

{'_id': ObjectId('652035aa8dddd3436e664723'), 'ride_id': 'C2F7DD78E82EC875', 'rideable_type': 'electric_bike', 'started_at': '2022-01-13 11:59:47', 'ended_at': '2022-01-13 12:02:44', 'start_station_name': 'Glenwood Ave & Touhy Ave', 'start_station_id': 525, 'end_station_name': 'Clark St & Touhy Ave', 'end_station_id': 'RP-007', 'start_lat': 42.0128005, 'start_lng': -87.665906, 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'member_casual': 'casual', 'started_at_date': '2022-01-13', 'started_at_time': '11:59:47', 'ended_at_date': '2022-01-13', 'ended_at_time': '12:02:44'}
{'_id': ObjectId('652045d1b7b2efde78396faa'), 'date': '2022-01-02', 'cloud_cover': 90.0, 'precipitation': 13.81, 'min_temp': 18.81, 'max_temp': 32.77, 'morning_temp': 28.24, 'afternoon_temp': 26.64, 'evening_temp': 19.63, 'night_temp': 32.77, 'max_windspeed': 18.41}
{'_id': ObjectId('652035aa8dddd3436e664723'), 'end_lat': 42.01256011541, 'end_lng': -87.6743671152, 'end_station_id': 'RP-007', 'end_station_name': 

In [9]:
# Use aggregation pipeline to find top ten start stations
pipeline = [
    {
        "$group": {
            "_id": "$start_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10StartStations"
}
]
# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign results to variable 
Top10StartStations = db['Top10StartStations']

In [10]:
# Count the number of documents in the collection 
print(Top10StartStations.count_documents({}))

10


In [11]:
# Review a document in the collection 
print(db.Top10StartStations.find_one())

{'_id': 'Streeter Dr & Grand Ave', 'count': 68245, 'latitude': 41.880958, 'longitude': -87.616743}


In [12]:
# Use aggregation pipeline to find top ten end stations
pipeline = [
    {
        "$group": {
            "_id": "$end_station_name",
            "count": {"$sum": 1},
            "latitude": {"$first": "$end_lat"},
            "longitude": {"$first": "$end_lng"}
        }
    },
    {
        "$sort": {"count": -1}
    },
    {
        "$limit": 10
    },
    {   "$out": "Top10EndStations"
}
]
# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10EndStations = db['Top10EndStations']

In [13]:
# Count the number of documents in the collection 
print(Top10EndStations.count_documents({}))

10


In [14]:
# Review a document in the collection 
print(db.Top10EndStations.find_one())

{'_id': 'Streeter Dr & Grand Ave', 'count': 69516, 'latitude': 41.892278, 'longitude': -87.612043}


In [15]:
# Create a pipeline query to find the top ten bike routes (by start and end station)
pipeline = [
    {
        "$group": {
            "_id": { "Start Station": "$start_station_name", "End Station": "$end_station_name"},
            "count": {"$sum": 1},
            "start latitude": {"$first": "$start_lat"},
            "start longitude": {"$first": "$start_lng"},
            "end latitude": {"$first": "$end_lat"},
            "end longitude": {"$first": "$end_lng"}
        }
    },
    {"$sort": {"count": -1}
},
    {
        "$limit": 10
},
    {   "$out": "Top10Routes"
}
]
# Perform the aggregation
result = list(withStation.aggregate(pipeline))

# Assign to a variable
Top10Routes = db['Top10Routes']

In [16]:
# Count the number of documents in the collection 
print(Top10Routes.count_documents({}))

10


In [17]:
# Review a document in the collection 
print(db.Top10Routes.find_one())

{'_id': {'Start Station': 'Streeter Dr & Grand Ave', 'End Station': 'Streeter Dr & Grand Ave'}, 'count': 11633, 'start latitude': 41.892278, 'start longitude': -87.612043, 'end latitude': 41.892278, 'end longitude': -87.612043}


In [18]:
# Create a pipeline query to find docouments that have lat/long  
pipeline = [
    {
        "$match": {
            "$and": [
                { "start_lat": { "$ne": "" } },
                { "start_lng": { "$ne": "" } },
                { "end_lat": { "$ne": "" } },
                { "end_lng": { "$ne": "" } }
            ]
        }
    }, 
    {"$out": "withLatLong"}
]

# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))

In [19]:
# Create a pipeline query to find distance of each route in descending order by length  
pipeline = [
    {
        "$addFields": {
            "start_lat": { "$toDouble": "$start_lat" },
            "start_lng": { "$toDouble": "$start_lng" },
            "end_lat": { "$toDouble": "$end_lat" },
            "end_lng": { "$toDouble": "$end_lng" }
        }
    },
    {
        "$addFields": {
            "distance": {
                "$sqrt": {
                    "$add": [
                        {
                            "$pow": [
                                { "$subtract": ["$end_lat", "$start_lat"] },
                                2
                            ]
                        },
                        {
                            "$pow": [
                                {
                                    "$multiply": [
                                        { "$subtract": ["$end_lng", "$start_lng"] },
                                        { "$cos": { "$avg": ["$start_lat", "$end_lat"] } }
                                    ]
                                },
                                2
                            ]
                        }
                    ]
                }
            }
        }
    },
    {
        "$sort": {"distance": -1}
    },
    {"$out": "RouteDistance"}
]

# Perform the aggregation
result = list(withLatLong.aggregate(pipeline))

# Assign to a variable
RouteDistance = db['RouteDistance']

In [20]:
# Count the number of documents in the collection 
print(RouteDistance.count_documents({}))

5429682


In [21]:
# Find the first 10 documents
documents = RouteDistance.find().sort("distance", -1).limit(10)

# Print the documents
for doc in documents:
    print(doc)

{'_id': ObjectId('65203787260559f00d1ed88b'), 'end_lat': 0.0, 'end_lng': 0.0, 'end_station_id': 'chargingstx07', 'end_station_name': 'Green St & Madison Ave*', 'ended_at': '2022-11-09 12:26:18', 'ended_at_date': '2022-11-09', 'ended_at_time': '12:26:18', 'member_casual': 'member', 'ride_id': 'E9495F1DC3475D41', 'rideable_type': 'classic_bike', 'start_lat': 41.884114, 'start_lng': -87.654264, 'start_station_id': 18062, 'start_station_name': 'Aberdeen St & Randolph St', 'started_at': '2022-11-09 12:21:55', 'started_at_date': '2022-11-09', 'started_at_time': '12:21:55', 'weather_data': {'_id': ObjectId('652045d1b7b2efde783970e1'), 'date': '2022-11-09', 'cloud_cover': 75.0, 'precipitation': 0.0, 'min_temp': 47.35, 'max_temp': 60.37, 'morning_temp': 51.44, 'afternoon_temp': 48.54, 'evening_temp': 51.37, 'night_temp': 51.82, 'max_windspeed': 13.8}, 'distance': 60.51865544715035}
{'_id': ObjectId('65203787260559f00d1eda8a'), 'end_lat': 0.0, 'end_lng': 0.0, 'end_station_id': 'chargingstx07', '

In [22]:
db["divvy_rides_by_month"].drop()

In [23]:
# Define the aggregation pipeline to pull rides by month 
pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": {"$toDate": "$started_at"}},
                "month": {"$month": {"$toDate": "$started_at"}}
            },
            "total_rides": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_rides": 1
        }
    },
    {
        "$sort": {"year": 1, "month": 1}
    }
]

# Execute the aggregation pipeline and write to a new collection
divvy_rides_by_month = db["divvy_rides_by_month"]
divvy_rides_by_month.drop()  # Drop the collection
aggregated_result = divvy_ridedata_merged.aggregate(pipeline, allowDiskUse=True, collation=None)

for doc in aggregated_result:
    divvy_rides_by_month.insert_one(doc)

print("Aggregation result has been written to the new collection.")

Aggregation result has been written to the new collection.


In [24]:
# Define the aggregation pipeline to define divvy rides by season 
pipeline = [
    {
        "$group": {
            "_id": {
                "year": "$year",
                "season": {
                    "$switch": {
                        "branches": [
                            {"case": {"$in": ["$month", [3, 4, 5]]}, "then": "Spring"},
                            {"case": {"$in": ["$month", [6, 7, 8]]}, "then": "Summer"},
                            {"case": {"$in": ["$month", [9, 10, 11]]}, "then": "Autumn"},
                            {"case": {"$in": ["$month", [12, 1, 2]]}, "then": "Winter"}
                        ],
                        "default": "Unknown"
                    }
                }
            },
            "total_rides": {"$sum": "$total_rides"}
        }
    },
    {
        "$sort": {"_id.year": 1, "_id.season": 1}
    }
]

# Execute the aggregation pipeline
divvy_rides_by_season = db["divvy_rides_by_season"]
divvy_rides_by_season.drop()  # Drop the collection
aggregated_result = list(divvy_rides_by_month.aggregate(pipeline, allowDiskUse=True, collation=None))

# Insert the aggregated documents into the new collection
for doc in aggregated_result:
    print("Inserting document:", doc)
    divvy_rides_by_season.insert_one(doc)

print("Aggregation by season result has been written to the new collection.")

Inserting document: {'_id': {'year': 2022, 'season': 'Autumn'}, 'total_rides': 1597759}
Inserting document: {'_id': {'year': 2022, 'season': 'Spring'}, 'total_rides': 1057778}
Inserting document: {'_id': {'year': 2022, 'season': 'Summer'}, 'total_rides': 2378624}
Inserting document: {'_id': {'year': 2022, 'season': 'Winter'}, 'total_rides': 401185}
Aggregation by season result has been written to the new collection.


In [25]:
print(db.list_collection_names())

['divvy_ridedata', 'withLatLong', 'Top10StartStations', 'withoutStationName', 'Top10Routes', 'withStationName', 'Top10EndStations', 'divvy_rides_by_season', 'divvy_ridedata_merged', 'RouteDistance', 'weather_daily', 'divvy_rides_by_month']


In [26]:
from bson import ObjectId

# Get distinct station names along with start_lat and start_lng
distinct_station_data = db["withStationName"].aggregate([
    {
        "$group": {
            "_id": "$start_station_name",
            "start_lat": {"$first": "$start_lat"},
            "start_lng": {"$first": "$start_lng"}
        }
    }
])

collection_name = "distinct_station_names"
station_names = db[collection_name]

station_name_documents = []
for data in distinct_station_data:
    station_name_documents.append({
        "start_station_name": data["_id"],
        "start_lat": data["start_lat"],
        "start_lng": data["start_lng"],
        "_id": str(ObjectId())
    })

station_names.insert_many(station_name_documents)

print(f"{len(station_name_documents)} distinct station names imported into '{collection_name}' collection.")

1635 distinct station names imported into 'distinct_station_names' collection.


In [27]:
print(distinct_station_names)

NameError: name 'distinct_station_names' is not defined

In [28]:
station_names.find_one()


{'_id': '65339541c6a39455604df43a',
 'start_station_name': 'Harborside',
 'start_lat': 40.7192517,
 'start_lng': -74.034234}

In [29]:
# Define the aggregation pipeline to pull rides by month 

# Use aggregation pipeline to create a collection that contains start and end station names
pipeline = [
         {"$match": {"weather_data.sig_prcp": {"$exists": True, "$eq": "yes"},
                     }},
         {"$out": "sig_prcp_yes"}
         
]
# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))
# Assign to a variable
sig_prcp_yes = db["sig_prcp_yes"]


In [30]:
# Define the aggregation pipeline to pull rides by month 

# Use aggregation pipeline to create a collection that contains start and end station names
pipeline = [
         {"$match": {"weather_data.sig_prcp": {"$exists": True, "$eq": "no"},
                     }},
         {"$out": "sig_prcp_no"}
         
]
# Perform the aggregation
result = list(divvy_ridedata_merged.aggregate(pipeline))
# Assign to a variable
sig_prcp_no = db["sig_prcp_no"]

In [31]:
# Create a new collection that shows average daily rides per month with precipitation 

pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": {"$toDate": "$started_at"}},
                "month": {"$month": {"$toDate": "$started_at"}}
            },
            "total_rides": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_rides": 1
        }
    },
    {
        "$sort": {"year": 1, "month": 1}
    }
]

# Execute the aggregation pipeline and write to a new collection
sig_prcp_yes_month = db["sig_prcp_yes_month"]
#divvy_rides_by_month = db["divvy_rides_by_month"]
sig_prcp_yes_month.drop()  # Drop the collection
aggregated_result = sig_prcp_yes.aggregate(pipeline, allowDiskUse=True, collation=None)

for doc in aggregated_result:
    sig_prcp_yes_month.insert_one(doc)

# Function to insert number of days and sig_prcp count for each month
def update_num_days_and_sig_prcp_count(year, month):
    _, num_days = monthrange(year, month)
    query = {'year': year, 'month': month}
    update_query = {'$set': {'num_days': num_days}}
    sig_prcp_yes_month.update_one(query, update_query)

    # Count the number of days with sig_prcp = 'no' for the given month
# Count the number of days with sig_prcp = 'no' for the given month
    sig_prcp_count = weather_daily.count_documents({
        'date': {'$regex': f'^{year:04d}-{month:02d}'},  # Match the year and month in the date field
        'sig_prcp': 'yes'
    })
    update_query = {'$set': {'sig_prcp_count': sig_prcp_count}}
    sig_prcp_yes_month.update_one(query, update_query)

    # Calculate average rides per day
    total_rides = sig_prcp_yes_month.find_one(query)['total_rides']
    sig_prcp_yes_month.update_one(query, update_query)

    # Calculate and update average rides per day with no significant precipitation
    query = {'year': 2022, 'month': month}
    document = sig_prcp_yes_month.find_one(query)
    
    if document['sig_prcp_count'] > 0:  # To avoid division by zero
        average_rides_per_day = document['total_rides'] / document['sig_prcp_count']
        update_query = {'$set': {'average_rides_per_day': average_rides_per_day}}
        sig_prcp_yes_month.update_one(query, update_query)

# Loop through each month in the year 2022
for month in range(1, 13):
    update_num_days_and_sig_prcp_count(2022, month)


TypeError: 'NoneType' object is not subscriptable

In [ ]:
# Create a new collection that shows average daily rides per month with no precipitation 

pipeline = [
    {
        "$group": {
            "_id": {
                "year": {"$year": {"$toDate": "$started_at"}},
                "month": {"$month": {"$toDate": "$started_at"}}
            },
            "total_rides": {"$sum": 1}
        }
    },
    {
        "$project": {
            "_id": 0,
            "year": "$_id.year",
            "month": "$_id.month",
            "total_rides": 1
        }
    },
    {
        "$sort": {"year": 1, "month": 1}
    }
]

# Execute the aggregation pipeline and write to a new collection
sig_prcp_no_month = db["sig_prcp_no_month"]
#divvy_rides_by_month = db["divvy_rides_by_month"]
sig_prcp_no_month.drop()  # Drop the collection
aggregated_result = sig_prcp_no.aggregate(pipeline, allowDiskUse=True, collation=None)

for doc in aggregated_result:
    sig_prcp_no_month.insert_one(doc)

def update_num_days_and_sig_prcp_count(year, month):
    _, num_days = monthrange(year, month)
    query = {'year': year, 'month': month}
    update_query = {'$set': {'num_days': num_days}}
    sig_prcp_no_month.update_one(query, update_query)

    # Count the number of days with sig_prcp = 'no' for the given month
    sig_prcp_count = weather_daily.count_documents({
        'date': {'$regex': f'^{year:04d}-{month:02d}'},
        'sig_prcp': 'no'
    })
    update_query = {'$set': {'sig_prcp_count': sig_prcp_count}}
    sig_prcp_no_month.update_one(query, update_query)

    # Calculate average rides per day
    document = sig_prcp_no_month.find_one(query)
    
    if document['sig_prcp_count'] > 0:
        total_rides = document['total_rides']
        average_rides_per_day = total_rides / document['sig_prcp_count']
        update_query = {'$set': {'average_rides_per_day': average_rides_per_day}}
        sig_prcp_no_month.update_one(query, update_query)

# Loop through each month in the year 2022
for month in range(1, 13):
    update_num_days_and_sig_prcp_count(2022, month)


In [ ]:
# Create a new collection to store documents with string _id
sig_prcp_no_month_string.drop()
sig_prcp_no_month_string = db["sig_prcp_no_month_with_string_id"]

# Iterate through the documents in the original collection
for document in sig_prcp_no_month.find({}):
    document_id = document['_id']
    string_id = str(document_id)
    
    # Create a new document with the string _id and other fields
    new_document = {
        '_id': string_id,
        'year': document['year'],
        'month': document['month'],
        'total_rides': document['total_rides'],
        'num_days': document['num_days'],
        'sig_prcp_count': document['sig_prcp_count'],
        'average_rides_per_day': document['average_rides_per_day']
        # Include other fields from the original document
    }
    
    # Insert the new document into the new collection
    sig_prcp_no_month_string.insert_one(new_document)

print("Documents with string _id inserted into the new collection.")

In [ ]:
# Create a new collection to store documents with string _id
sig_prcp_yes_month_string.drop()
sig_prcp_yes_month_string = db["sig_prcp_yes_month_with_string_id"]

# Iterate through the documents in the original collection
for document in sig_prcp_yes_month.find({}):
    document_id = document['_id']
    string_id = str(document_id)
    
    # Create a new document with the string _id and other fields
    new_document = {
        '_id': string_id,
        'year': document['year'],
        'month': document['month'],
        'total_rides': document['total_rides'],
        'num_days': document['num_days'],
        'sig_prcp_count': document['sig_prcp_count'],
        'average_rides_per_day': document['average_rides_per_day']
        # Include other fields from the original document
    }
    
    # Insert the new document into the new collection
    sig_prcp_yes_month_string.insert_one(new_document)

print("Documents with string _id inserted into the new collection.")

In [32]:
# Retrieve all documents from the collection
all_documents = sig_prcp_yes_month_string.find({})

# Iterate through the documents and print them
for doc in all_documents:
    print(doc)

NameError: name 'sig_prcp_yes_month_string' is not defined

In [ ]:
# Retrieve all documents from the collection
all_documents = sig_prcp_no_month_string.find({})

# Iterate through the documents and print them
for doc in all_documents:
    print(doc)